In [ ]:
!nvidia-smi

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# inspect data
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

## Creating data loaders (prepering the data)

In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images:")
train_data_10_prec = train_datagen.flow_from_directory(train_dir,
                                                       target_size=IMAGE_SHAPE,
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")

print("Test Images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                                       target_size=IMAGE_SHAPE,
                                                       batch_size=BATCH_SIZE,
                                                       class_mode="categorical")

## Setting up callbacks

In [ ]:
# Create Tensorboard callback
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## Creating models using TensorFlow Hub

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import Dep
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Create model function
def create_model(model_url, num_classes=10):
  """
  model_url: TF Hub url for feature extraction
  num_classes: num of neurons in output layer

  returns uncompiled TF model
  """
  # Download the preload model
  feature_extractor_layer  = hub.KerasLayer(model_url,
                                            trainable=False,
                                            name="feature_extraction_layer",
                                            input_shape=IMAGE_SHAPE+(3,))
  
  # Create model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation="softmax", name="output_model")
  ])

  return model

## Creating ResNet Model


In [ ]:
# create resnet model
resnet_model = create_model(resnet_url,
                            num_classes=train_data_10_prec.num_classes)

In [ ]:
resnet_model.summary()

In [ ]:
# Compile a  resnet model
resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
history_resnet = resnet_model.fit(
    train_data_10_prec,
    epochs=5,
    steps_per_epoch=len(train_data_10_prec),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                           experiment_name="resnet50V2")]

)

In [ ]:
from matplotlib import pyplot as plt

# Plot the validation and training curves separately
def plot_loss_curves(history):
  """
  Returns separate loss curves
  """
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.figure(figsize=(18,7))

  plt.subplot(1,2,1)
  plt.plot(epochs, loss, label="Training Loss")
  plt.plot(epochs, val_loss, label="Validation Loss")
  plt.title("loss")
  plt.xlabel("epochs")
  plt.legend()

  # Plot Accuracy
  plt.subplot(1,2,2)
  plt.plot(epochs, accuracy, label="Training Accuracy")
  plt.plot(epochs, val_accuracy, label="Validation Accuracy")
  plt.title("accuracy")
  plt.xlabel("epochs")
  plt.legend();

In [ ]:
plot_loss_curves(history_resnet)

## Creating EfficientNet model

In [ ]:
effnet_model = create_model(efficientnet_url, train_data_10_prec.num_classes)

In [ ]:
effnet_model.summary()

In [ ]:
# Compiling the model
effnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
history_effnet = effnet_model.fit(
    train_data_10_prec,
    epochs=5,
    steps_per_epoch=len(train_data_10_prec),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                           experiment_name="efficientnetB0")]
)

In [ ]:
plot_loss_curves(history_effnet)

## Comparing models on TensorBoard

In [ ]:
# Upload tensorboards dev records
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EffNetB0 vs ResNet50v2" \
  --description "Comparing image classification models" \
  --one_shot

In [ ]:
!tensorboard dev list